# Analyse MARC (https://marc.ifremer.fr) data 

In [ ]:
import fsspec
import intake
import xarray as xr

catalogue="/home/datawork-taos-s/intranet/kerchunk/ref-marc.yaml"
catalogue = "https://data-taos.ifremer.fr/kerchunk/ref-marc.yaml"

In [ ]:
def list_param(cat, param):
    return cat.metadata["parameters"][param]["allowed"]

def allowed_param(cat):
    return cat.metadata["parameters"]

def allowed_years(region):
    fs = fsspec.filesystem("https") if ('https' in catalogue) else fsspec.filesystem("file")
    files = fs.glob(catalogue.replace(".yaml", "/") + region + "/*.json.zst")
    years = [
        f.replace(".json.zst", "").replace(
            catalogue.replace(".yaml", "/") + region + "/", ""
        )
        for f in files
    ]
    years.sort()
    return years

def year_concat(cat, source, region):
    concat = [
        cat[source](region=region, year=i).to_dask() for i in allowed_years(region)
    ]

    return xr.concat(
        concat, dim="time", coords="minimal", data_vars="minimal", compat="override"
    )

Detail of the functions:
-----------------------------------------------------------------------------------------------  
    list_param(cat, param)    
>- Takes 2 arguments: cat = An opened catalog, param = The name of a parameter.
>- Returns a ptyhon list containing all the parameters.

    allowed_param(cat)    
>- Takes 1 argument: cat = An opened catalog
>- Returns a python dict containing the allowed values of each parameters and their description

    Allowed_years(region)    
>- Takes 1 argument: region = A region of data.
>- Returns a pythton list containing all the year available in kerchunk catalogue.  

    year_concat(cat, source, region)    
>- Takes 3 arguments: cat = An opened catalog, source = the source from this catalog, region = the region where the data comes from
>- Returns: A dataset concatenated over the all available in kerchunk. 


# Open intake catalgue.  

In [ ]:
cat = intake.open_catalog(catalogue)

## Inspect intake catalogue

In [ ]:
regions = list_param(cat, "region")

In [ ]:
regions

## Define a region, inspect the catalogue to know which year the datas are availableand, then load the data

In [ ]:
region_local = "f1_e2500_agrif/MARC_F1-MARS3D-SEINE"

In [ ]:
allowed_years(region_local)

In [ ]:
region_global = "f1_e2500_agrif/MARC_F1-MARS3D-MANGAE2500-AGRIF"

In [ ]:
allowed_years(region_global)

In [ ]:
year='2018'
time=['2018-06-23T00:00:00.000008', '2018-06-23T12:00:00.000000']
time='2018-06-23'

In [ ]:
ds_local = cat.marc(region=region_local , year=year).to_dask().sel(time=time)
ds_global = cat.marc(region=region_global, year=year).to_dask().sel(time=time)

## Plot data

In [ ]:
%%time
da_local = ds_local.TEMP.unify_chunks()
da_global = ds_global.TEMP.unify_chunks()



In [ ]:
#import geoviews.feature as gf
import hvplot.xarray

(
da_local.hvplot.quadmesh(
    x="longitude", y="latitude", rasterize=True, geo=True, cmap="BuPu", coastline=True
)

 ) 
#* gf.land.opts(scale="10m") * gf.coastline.opts(scale="10m")

In [ ]:
da_global.hvplot.quadmesh(
    x="longitude", y="latitude", rasterize=True, geo=True, cmap="BuPu", coastline=True
)